In [1]:
import zipfile
import pandas as pd
from collections import Counter
import re
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

In [2]:
df = "/ctc/DATASTORE/TwitterReleases/InformationOperations/202006/hashed_2020_05_turkey_052020_turkey_052020_tweets_csv_hashed.zip"
with zipfile.ZipFile(df, 'r') as zObject:
    # List the contents of the zip file
    for member in zObject.namelist():
        if member.endswith(".csv"):
            df = pd.read_csv(zObject.open(member))


df = df[df['user_mentions'].notna()]  # Filter out rows with NaN values in 'user_mentions' column
mentions = df['user_mentions'].tolist()  # Convert 'user_mentions' column to a list



/tmp/ipykernel_356506/335624985.py:6: DtypeWarning: Columns (15,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(zObject.open(member))
/tmp/ipykernel_356506/335624985.py:6: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(zObject.open(member))
/tmp/ipykernel_356506/335624985.py:6: DtypeWarning: Columns (15,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(zObject.open(member))
/tmp/ipykernel_356506/335624985.py:6: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(zObject.open(member))
/tmp/ipykernel_356506/335624985.py:6: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(zObject.open(member))
/tmp/ipykernel_356506/335624985.py:6: DtypeWarning: Columns (15,19) have mixed types. Specify 

In [3]:
cleaned_mentions = []
for mention_list in mentions:
    mention_list = re.sub(r"[^\w\s,]", "", mention_list)  # Remove non-alphanumeric characters except commas
    mention_list = mention_list.split(",")  # Split the string by commas
    mention_list = [mention.strip().lower() for mention in mention_list if mention.strip() != ""]  # Clean and filter mentions
    cleaned_mentions.append(mention_list)

df['cleaned_user_mentions'] = cleaned_mentions  # Create a new column with cleaned mentions
filtered_df = df[df['tweet_text'].str.contains("erdogan|akp", case=False) & df['cleaned_user_mentions'].apply(lambda x: bool(set(x) & set(["1847174592","1482806160","3555190636","466757008","442252059","1132163245","4409222427","876745005561466880","3291835017","1631039467"])))]


In [4]:
filtered_df

,tweetid,userid,user_display_name,user_screen_name,user_reported_location,user_profile_description,user_profile_url,follower_count,following_count,account_creation_date,...,latitude,longitude,quote_count,reply_count,like_count,retweet_count,hashtags,urls,user_mentions,cleaned_user_mentions


In [5]:
import string
#!pip3 install zemberek-python


from zemberek import (
    TurkishSpellChecker,
    TurkishSentenceNormalizer,
    TurkishSentenceExtractor,
    TurkishMorphology,
    TurkishTokenizer
)

zemberek_tokenizer = TurkishTokenizer.DEFAULT

In [6]:
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r' ',text)

def text_preprocess(text):
    allowed_types = ['WordWithSymbol', 'Word', 'Punctuation', 'Number', 'UnknownWord']
    translator = str.maketrans(string.punctuation.replace('.','').replace('@',''), ' '*len(string.punctuation.replace('.','').replace('@','')))
    
#    text = deEmojify(text)
    text = " ".join([token.content for token in zemberek_tokenizer.tokenize(text) if token.type_.name in allowed_types])
    text = text.translate(translator)
    text = re.sub(' +', ' ', text)
    text = [word for word in text.split()]
    return " ".join(text)

In [7]:
filtered_df['processed_text'] = df.tweet_text.apply(lambda x: text_preprocess(x))


/tmp/ipykernel_356506/706462811.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['processed_text'] = df.tweet_text.apply(lambda x: text_preprocess(x))


In [8]:
filtered_df.processed_text[:10]

0                              bu benim hayat felsefem
1    Günaydın hayırlı günler Dünyadaki en guzel duy...
2                                                     
3    Oh be . 5 sorudan 3 ünü çözdüm . Eğer hepsi do...
4    Tek istedigim birlikte yüzlerce fotografimiz o...
5    Eygi bugün ne yazmış Modern Türkiye Batarmış A...
6    Aşk celladindan ne çikar madem ki yar vardır ....
7    CeHaPe ile hizmet farkımız Onlar kafa çekerek ...
8    O sadece KÜRTÇE albüm çıkartıcam dedi ve şeref...
9        yarın sınavım var ve ben çalışmak istemiyorum
Name: processed_text, dtype: object

In [9]:
#!pip3 install transformers
#!pip3 install torch
#!pip3 install torch torchvision
#!pip3 install xformers

from transformers import AutoModel, AutoTokenizer,pipeline, AutoModelForSequenceClassification 


model_2 = AutoModelForSequenceClassification.from_pretrained("kuzgunlar/electra-turkish-sentiment-analysis")
tokenizer_2 = AutoTokenizer.from_pretrained("kuzgunlar/electra-turkish-sentiment-analysis")

sa_2 = pipeline("sentiment-analysis", tokenizer=tokenizer_2, model=model_2)


def sentiment_analyzer(text):
    result = sa_2(text)
    
    if result[0]['score'] <= 0.7:
        sentiment = 0
    
    else:
        if result[0]['label'] == 'Positive':
            sentiment = 1

        else:
            sentiment = -1
            
    return sentiment

/cta/users/uduygu/project1/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
def sentiment_label(text):
    score = sa_2(text)
    label = score[0]['label']
    return label

def sentiment_score(text):
    score = sa_2(text)
    score = score[0]['score'] 
    return score


In [11]:
#specific charchter tweets
#filtered_df['processed_text'] = filtered_df.tweet_text.apply(lambda x: text_preprocess(x))

filtered_df['sentiment_label'] = filtered_df.processed_text.apply(lambda x: sentiment_label(x))
filtered_df['sentiment_score'] = filtered_df.processed_text.apply(lambda x: sentiment_score(x))



/tmp/ipykernel_356506/2369780362.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['sentiment_label'] = filtered_df.processed_text.apply(lambda x: sentiment_label(x))
/tmp/ipykernel_356506/2369780362.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['sentiment_score'] = filtered_df.processed_text.apply(lambda x: sentiment_score(x))


In [12]:
filtered_df['sentiment_label'].value_counts()


sentiment_label
Positive    144362
Negative    106327
Name: count, dtype: int64

# sentiment for top 10 atatürk including nodes from mentions network

In [13]:
filtered_df['sentiment_score'].mean()


0.91872140057238